In [1]:
import pennylane as qml
from pennylane import AdamOptimizer
import pennylane.numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
X_train = pd.read_csv("Data/balanceado/x_train.csv")
y_train = pd.read_csv("Data/balanceado/y_train.csv")
X_test = pd.read_csv("Data/balanceado/x_test.csv")
y_test = pd.read_csv("Data/balanceado/y_test.csv")

In [3]:
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_train = np.tensor(scaler.fit_transform(X_train), requires_grad=False)
y_train = np.tensor(y_train, requires_grad=False)
X_test = np.tensor(scaler.fit_transform(X_test), requires_grad=False)
y_test = np.tensor(y_test, requires_grad=False)

In [4]:
print("Formato pronto para o circuito:", X_train.shape)
print("Primeira amostra:", X_train[0])

Formato pronto para o circuito: (398041, 31)
Primeira amostra: [2.2115077  2.2115077  1.71899362 2.26621085 0.44272051 2.40974035
 1.44882717 1.3687728  0.9130568  2.03672787 0.37682694 0.91067877
 2.7957213  0.08980247 1.86495179 0.10669659 0.1831735  0.06342473
 0.39947921 0.54387861 1.63946035 2.47736155 2.25215233 1.13429557
 1.51489267 0.60352146 1.723606   1.80544441 2.3639359  1.49812174
 1.89821665]


In [5]:
y_test = y_test * 2 - 1
y_train = y_train * 2 - 1

In [6]:
y_train = y_train[:, 1]
y_test = y_test[:, 1]

In [7]:
from sklearn.model_selection import train_test_split


X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test,
    test_size=0.5,
    random_state=42,
    shuffle=True
)


In [8]:
dev = qml.device("lightning.qubit")

In [23]:
def layer(weights_da_camada, n_qubits):
    # weights_da_camada.shape = (n_qubits, 3)

    for i in range(n_qubits):
        qml.RY(weights_da_camada[i, 1], wires=i)

    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    for i in range(n_qubits):
        qml.RY(weights_da_camada[i, 1], wires=i)


n_qubits = 2  # por exemplo

@qml.qnode(dev)
def circuito(weights, x):
    qml.AmplitudeEmbedding(
        features=x,
        wires=range(n_qubits),
        pad_with=0.0,
        normalize=True
    )

    for layer_w in weights:
        layer(layer_w, n_qubits)

    return qml.expval(qml.PauliZ(0))


In [24]:
def cost(weights, bias, X, Y):
    raw_preds = [circuito(weights, x) + bias for x in X]
    predictions = qml.math.stack(raw_preds)

    probs = 1 / (1 + qml.math.exp(-predictions))

    loss = -qml.math.mean(
        Y * qml.math.log(probs + 1e-7)
        + (1 - Y) * qml.math.log(1 - probs + 1e-7)
    )
    return loss


In [25]:
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

In [26]:
n_layers = 2
shape_weights = (n_layers, n_qubits, 3)
weights_init = 0.01 * np.random.randn(n_layers,n_qubits,3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

In [28]:
opt = qml.AdamOptimizer(0.001)

In [31]:
def pad_features(X):
    X_pad = np.zeros((X.shape[0], 4))
    X_pad[:, :2] = X
    return X_pad

X_pad = pad_features(X)
X_pad = X_pad / np.linalg.norm(X_pad, axis=1, keepdims=True)
y = y.astype(float)
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_pad, y, test_size=0.2, random_state=42
)


In [32]:
weights = weights_init
bias = bias_init
epochs = 50
batch_size = 30 # Exemplo
from tqdm import tqdm
pbar = tqdm(total=epochs, desc="Epoch")
for it in range(epochs):
    # Amostragem do Batch
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = y_train[batch_index]

    # Passo de Otimização
    # Note: passamos X e Y com as letras maiúsculas para bater com a def cost(..., X, Y)
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # --- OTIMIZAÇÃO DE LOGS ---
    # Só calcula a acurácia total a cada 10 passos para não travar o PC
    current_predictions = [np.sign(circuito(weights, x) + bias) for x in X_train]

    current_cost = cost(weights, bias, X_batch, Y_batch)

    acc = accuracy(y_train, current_predictions)
    pbar.update(1)
    pbar.set_postfix(loss=current_cost)


Epoch: 100%|██████████| 50/50 [05:58<00:00,  7.17s/it, loss=0.8640506658877779]

Epoch: 100%|██████████| 50/50 [01:55<00:00,  1.77s/it, loss=0.7708518703652375]